# SETUP

# TODO
- Aggiungere cella per diminuire calcoli (e dimensione dataset?)
- Creare un RDD i cui elementi siano le coppie righe/colonne e con un'operazione di map/reduce emettere coppie chiave-valore nelle quali la chiave indica la posizione (i, j) dell'elemento della matrice da aggiornare e il valore indica il nuovo elemento della matrice, utilizzando le formule del paragrafo 9.4.4. In questo modo "parallelizzate" gli aggiornamenti di tutti gli elementi della matrice. Eventualmente potete evitare di aggiornare d'un colpo tutta la matrice (potrebbe portare a difficoltà nella convergenza), decidendo in modo probabilistico ogni volta se aggiornare oppure no un elemento: basta che il job map/reduce generi un bit pseudocasuale che vale 1 con probabilità p, e se il suo valore è 1 allora utilizzate le formule di cui sopra, altrimenti restituite il valore attuale di U (fa parte della colonna che ottenete in input). In questo modo aggiornerete all'incirca una frazione pari a p dei valori della matrice. Lo stesso vale per V.

In [ ]:
if "google.colab" in str(get_ipython()):
    !pip install pyspark
    !git clone https://github.com/lukebella/YelpRecommenderSystem.git
    !mv YelpRecommenderSystem/* .
    !rm -fr YelpRecommenderSystem


In [ ]:
import os

os.environ['KAGGLE_USERNAME'] = "xxxxxx"
os.environ['KAGGLE_KEY'] = "xxxxxx"
!kaggle datasets download -p ./data -d yelp-dataset/yelp-dataset
!unzip -n ./data/yelp-dataset.zip -d ./data


In [ ]:
review_filename = 'data/yelp_academic_dataset_review.json'
user_filename = 'data/yelp_academic_dataset_user.json'
business_filename = 'data/yelp_academic_dataset_business.json'

# Lavorare su un sample più piccolo (10%)
### conviente farlo solo sulle review per problemi di join

In [ ]:
from pyspark.sql import SparkSession

# .master().config('spark.driver.memory', "15g")
spark = SparkSession.builder.getOrCreate()
# sc = pyspark.SparkContext().getOrCreate()
sc = spark.sparkContext


In [ ]:
# TODO add docstring
def from_json_to_RDD(filename):
    raw_df = spark.read.json(filename)
    raw_df.show()
    #return raw_df.sample(frac=1/10, random_state = 0).dropna().rdd
    return raw_df.rdd

raw_review_RDD = from_json_to_RDD(review_filename)#.sample(True, 1, 0)  #699192 rows #shuffle
raw_user_RDD = from_json_to_RDD(user_filename)
raw_business_RDD = from_json_to_RDD(business_filename)
#print(raw_business_RDD.filter(lambda x: x[2]=='bqFG0QJY9jj2m55OqAVHeA').first())

raw_review_RDD.first()

In [ ]:
#sample number of users and take all the reviews made by the users in the subset
#TODO that not all the users taken have made a review, we could try to eliminate those who have not made a review? 
# Those users should see the starting recommendation...
print(raw_user_RDD.count())
raw_user_RDD = raw_user_RDD.filter(lambda x:x[-4]>0).sample(False, 1/500, 0) #withReplacement of .sample True or False?
user_list = raw_user_RDD.map(lambda x: x[-2]).collect()
#3295
raw_review_RDD = raw_review_RDD.filter(lambda x: (x[8] in user_list))
print(len(user_list))
raw_review_RDD.count()
#11444


# RETRIEVE RDDs

In [ ]:
def get_review_tuple(entry):
    """ Parse a row in the review dataset form pyspark.sql.Row to tuple (remove unintersted columns)
    Args:
        entry (pyspark.sql.types.Row): a row in the review dataset in JSON format
    Returns:
        tuple: (review_id, user_id, business_id, stars, useful, funny, cool, text)
    """

    return (str(entry["review_id"]),    # 0
            str(entry["user_id"]),      # 1
            str(entry["business_id"]),  # 2
            int(entry["stars"]),        # 3
            int(entry["useful"]),       # 4
            int(entry["funny"]),        # 5
            int(entry["cool"]),         # 6
            str(entry["text"]))         # 7


review_RDD = raw_review_RDD.map(get_review_tuple)

review_count = review_RDD.count()

print(f'There are {review_count} reviews in the dataset')
print(f'Reviews: {review_RDD.first()}')


In [ ]:
def get_user_tuple(entry):
    """ Parse a row in the user dataset form pyspark.sql.Row to tuple (remove unintersted columns)
    Args:
        entry (pyspark.sql.types.Row): a row in the user dataset in JSON format
    Returns:
        tuple: (user_id, name, review_count, average_stars, useful, funny, cool, fans)
    """

    return (str(entry["user_id"]),          # 0
            str(entry["name"]),             # 1
            int(entry["review_count"]),     # 2
            float(entry["average_stars"]),  # 3
            int(entry["useful"]),           # 4
            int(entry["funny"]),            # 5
            int(entry["cool"]),             # 6
            int(entry["fans"]))             # 7


user_RDD = raw_user_RDD.map(get_user_tuple)

user_count = user_RDD.count()

print(f'There are {user_count} users in the dataset')
print(f'Users: {user_RDD.first()}')


In [ ]:
def get_business_tuple(entry):
    """ Parse a row in the business dataset form pyspark.sql.Row to tuple (remove unintersted columns)
    Args:
        entry (pyspark.sql.types.Row): a row in the business dataset in JSON format
    Returns:
        tuple: (business_id, name, city, state, stars, review_count, categories)
    """

    categories = [] if entry["categories"] == None \
                    else str(entry["categories"]).split(", ")
    
    return (str(entry["business_id"]),  # 0
            str(entry["name"]),         # 1
            str(entry["city"]),         # 2
            str(entry["state"]),        # 3
            float(entry["stars"]),      # 4
            int(entry["review_count"]), # 5
            categories)                 # 6

#TODO Attributes?

business_RDD = raw_business_RDD.map(get_business_tuple)

business_count = business_RDD.count()
# print(business_RDD.filter(lambda x: x[0]=='1H8ReY5GlGcHJz7umVidkg').first())

print(f'There are {business_count} business in the dataset')
print(f'Business: {business_RDD.first()}')

# Calculate overall
### - for each business_id:

$$
  \Delta = \dfrac{1}{2} 
              \left( 
                  \dfrac{\text{useful} + \dfrac{1}{2}(\text{funny} + \text{cool})} 
                        {\text{best\ useful} + \dfrac{1}{2}(\text{best\ funny} + \text{best\ cool})}
                        + 
                  \dfrac{\text{fans}}
                        {\text{best\ fans}}
              \right)
$$
$$
\Delta : [0, 1]
$$
$$
  \text{overall} = \begin{cases} 
              \text{stars} + \Delta & \text{if stars } \ge 3\\ 
              \text{stars} - \Delta & \text{if stars } \lt 3
            \end{cases}
$$

In [ ]:
# TODO fix all comment

#review_RDD.filter(lambda x: x[3]==0).count()


#stars - useful - funny - cool - numero fan


#if stars <3
    #stars - {[useful + (funny + cool)/2]/[best useful + (best funny + best cool)/2] + (numero fan/best user fans)}/2
#else
    #stars + {[useful + (funny + cool)/2]/[best useful + (best funny + best cool)/2] + (numero fan/best user fans)}/2

    #dati mancanti: numero fan; best fan; best useful/funny/cool per ristorante
#rdd_test_ufc = (id ristorsante, best useful, best funny, best cool)
#best fan = query su dataset utenti --> user_RDD.max().first()

#review_id - user_id - id_rist - overall


In [ ]:
# get_overall_rating: function that generates the bias in order to incentivate or decrease the importance or overall rating of a review

# TODO fix function

#+ (fans/best_user_fans)) / 2

# def get_overall_rating(stars, best_useful, best_funny, best_cool, useful, funny, cool):
#     overall = (useful + (funny + cool)/2) / (best_useful + (best_funny + best_cool)/2) 
#     if stars < 3:
#         return stars - overall
#     else:
#         return stars + overall


In [ ]:
def get_business_id_useful_funny_cool(row):
    """
        Writes a tuple with key business_id and attributes useful, funny, cool
    """
    return (row[2], (row[4], row[5], row[6]))

def get_max_useful_funny_cool(row1, row2):
    """
        Returns a new RDD with, for each business_id key, the maxima values of useful, funny, cool
    """
    return tuple(max(row1[i], row2[i]) for i in range(3))

review_tuple_RDD = review_RDD.map(get_business_id_useful_funny_cool)
review_best_ufc_RDD = review_tuple_RDD.reduceByKey(get_max_useful_funny_cool)

review_best_ufc_RDD.take(5)

# review_best_ufc_RDD tuple: (business_id, (useful, funny, cool)): for each reastaurant, this tuple takes the maxima values of useful, funny and cool. 


In [ ]:
def rearrange_review_and_best_ufc_RDD(row):
    """
        Merge, for each business_id, the row values with the best useful, funny, cool ones
    """
    return (row[2], (row[0], row[1], row[3], row[4], row[5], row[6]))

review_ufc_RDD = review_RDD.map(rearrange_review_and_best_ufc_RDD).join(review_best_ufc_RDD)
review_ufc_RDD.first()

# (id_business, ((id_review, id_user, star, useful, funny , cool), (best useful, best funny, best cool)))


In [ ]:
def partial_overall(x):
    """
        Rearrange review_ufc_RDD and calculates the partial overall of the delta above
        partial_overall ={[useful + (funny + cool)/2]/[best useful + (best funny + best cool)/2]
    """
    key = (x[1][0][0], x[1][0][1], x[0], x[1][0][2])
    num = (x[1][0][3] + (x[1][0][4] + x[1][0][5])/2)
    den = (x[1][1][0] + (x[1][1][1] + x[1][1][2])/2)
    return (*key,(num/den if sum(x[1][1]) != 0 else 0))


partial_review_overall_RDD = review_ufc_RDD.map(partial_overall)
partial_review_overall_RDD.first()

# tuple: (review_id, user_id, buisness_id, stars, partial overall) 

In [ ]:
# tuple: (user_id, fans)
user_fans_RDD = user_RDD.map(lambda x: (x[0], x[7]))

# TODO add map to turn the result in a simple tuple
# tuple: (user_id, (review_id, buisness_id, stars, partial overall), fans)
partial_overall_fans_RDD = partial_review_overall_RDD.map(lambda x: (x[1], (x[0], x[2], x[3], x[4]))).join(user_fans_RDD)

partial_overall_fans_RDD.first()


In [ ]:
# user with the highest number of fans
best_user_fans = user_RDD.max(lambda x: x[7])
print("User with the highest number of fans: ",best_user_fans)

# TODO if best user fans = 0

def review_overall(x):
    user_id, review_id, business_id, stars= x[0], x[1][0][0], x[1][0][1], x[1][0][2]
    value = stars + (x[1][0][3]+x[1][1]/best_user_fans[7])/(2 if stars >=3 else -2) 
    return (review_id, user_id, business_id, value)

    #tuple: (review_id, user_id, business_id, stars + (partial overall + fans/best_fans))


review_overall_RDD = partial_overall_fans_RDD.map(review_overall)

review_overall_RDD.sortBy(lambda x:x[3], ascending=False).take(5)


# Basic Recommendation

In [ ]:
#Keeping businesses that have at least 20 reviews

#review_by_business_filtered_RDD = review_overall_RDD.map(lambda x: (x[2], (x[0], x[1], x[3]))).groupByKey()\
#                                                    .mapValues(list).filter(lambda x: len(x[1])>=20)
# review_by_business_filtered_RDD.first()


In [ ]:
def sortFunction(tuple):
    """ Construct the sort string (does not perform actual sorting)
    Args:
        tuple: (rating, MovieName)
    Returns:
        sortString: the value to sort with, 'rating MovieName'
    """
    #key = unicode('%.3f' % tuple[0])
    if (tuple[1][1]!= None):
        value = '{:.3f}'.format(tuple[1][1])
    else:
        value = ''
    key = tuple[0]
    return (value + ' ' + key)


In [ ]:
#Make, for each business, the average of the stars for all the reviews

# business_overall_RDD = review_by_business_filtered_RDD.map(lambda x: (x[0],  sum(i[2] for i in x[1])/len(x[1])))
# business_overall_RDD.first()

In [ ]:
# top recommended businesses
#business_RDD.map(lambda x: (x[0], (x[1], x[6]))).leftOuterJoin(business_overall_RDD).sortBy(sortFunction, False).take(10)  #x[2], x[3],.filter(lambda x: 'Restaurants' in x[1][1])


# Collaborative Filtering

In [ ]:
review_overall_bis = review_overall_RDD.map(lambda x: (x[1], x[2], x[3]))


In [ ]:
#Cosine Distance
def cosine_dist(value):
    """ 
    arg: utente da confrontare
    per ogni utente:
        - user_id, prodotto tra rating di item per stessa chiave di item, lista di rating
        - somma di valori nel numeratore, somma di liste del denominatore
        - prodotto delle somma quadratiche nel denominatore tra i due utenti
        - prendi utenti che hanno qualche voto su un business in comune diverso da zero
    """
    value_review = dict(review_overall_bis.filter(lambda x:x[0] == value).map(lambda x:(x[1],x[2])).collect()) #business: voto
    user_sim = review_overall_bis.map(lambda x: (x[0], (0 if x[1] not in value_review.keys() \
                                                        else x[2]*value_review[x[1]], [x[2]])))\
                                 .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
                                 .map(lambda x:(x[0], x[1][0]/((sum(i**2 for i in x[1][1])**0.5) * \
                                                               (sum(i**2 for i in value_review.values())**0.5))))\
                                 .filter(lambda x:x[1]!=0 and x[0]!=value).take(5)
    return user_sim, value_review

def collaborative_filtering(value):
    print(value)
    user_sim, value_review = cosine_dist(value)
    print(value_review.keys())
    return review_overall_bis.filter(lambda x: (x[0] in dict(user_sim).keys() and x[1] not in value_review.keys()))\
                             .sortBy(lambda x:x[2],ascending = False).take(10) 


In [ ]:
# print(review_overall_bis.take(10)[5])
business_collab = collaborative_filtering('onQsolCnpHwtOZCIuI91vQ')
# print(review_overall_RDD.filter(lambda x:x[1]=='Eep1pCr1zMqub16tYgUFgQ').collect())
business_collab


# UV Decompostion

In [ ]:
#TODO in the collaborative prt, take into consideration the categories and the State
#TODO based on the number of reviews, multiply by the percentage of the reviews made in a particular state (same for the categories)
# print(dict(sorted(business_RDD.flatMap(lambda x: tuple(x[6])).countByValue().items(), key=lambda x: x[1], reverse=True))) lista delle categorie più frequenti


## Preprocessing

In [ ]:
# Preprocessing
# Because the differences in the quality of items and the rating scales of users are
# such important factors in determining the missing elements of the matrix M , it
# is often useful to remove these influences before doing anything else. The idea
# was introduced in Section 9.3.1. We can subtract from each nonblank element
# mij the average rating of user i. Then, the resulting matrix can be modified
# by subtracting the average rating (in the modified matrix) of item j. It is also
# possible to first subtract the average rating of item j and then subtract the
# average rating of user i in the modified matrix. The results one obtains from
# doing things in these two different orders need not be the same, but will tend
# to be close. A third option is to normalize by subtracting from mij the average
# of the average rating of user i and item j, that is, subtracting one half the sum
# of the user average and the item average.
# If we choose to normalize M , then when we make predictions, we need to
# undo the normalization. That is, if whatever prediction method we use results
# in estimate e for an element mij of the normalized matrix, then the value
# we predict for mij in the true utility matrix is e plus whatever amount was
# subtracted from row i and from column j during the normalization process.



#We have chosen to implement the third option
#review_overall_RDD.flatMap(lambda x : ((x[1], x[2]), 1)).reduceByKey(lambda x, y : x + y).sortBy(lambda x: x[1], False)


In [ ]:
#media voti utente
#average_user_RDD = review_overall_RDD.map(lambda x: (x[1], x[3])).reduceByKey(lambda x,y: (x+y)/2)
# average_user_RDD.take(10)


In [ ]:
#media voti business
#
#average_business_RDD = review_overall_RDD.map(lambda x: (x[2], x[3])).reduceByKey(lambda x,y: (x+y)/2)
# average_business_RDD.take(10)


In [ ]:
#Merge review rates with average user
#partial_normalized_RDD = review_overall_RDD.map(lambda x: (x[1], (x[0], x[2], x[3]))).join(average_user_RDD).map(lambda x: (x[1][0][0], x[0], x[1][0][1], x[1][0][2], x[1][1]))
# partial_normalized_RDD.take(10)

# tuple: (review_id, user_id, buisness_id, overall, average_user)


In [ ]:
# normalized_RDD = partial_normalized_RDD.map(lambda x: (x[1], (x[0], x[2], x[3]))) \
#                                        .join(average_business_RDD) \
#                                        .map(lambda x: ((x[1][0][0], x[0]), (x[1][0][1] - (x[1][1]+x[1][0][2])/2))) \
#                                        .reduceByKey(lambda x, y : (x+y)/2) \
#                                        .map(lambda x: (*x[0], x[1]))

# normalized_RDD.first()
# # tuple: (user_id, buisness_id, overall - normalized_value)


## Initialization

In [ ]:
#We need to train and predict the vectors U and V
# train(trainingRDD, rank, iterations, regularization_parameter)  --> (U,V)
#We need to calculate RMSE between the utility and our matrix

#TODO initialize U and V with the overall average value

rank = 2
# regularization_parameter = 0.1

U_RDD = user_RDD.map(lambda x: (x[0], [1 for _ in range(rank)]))#.sortBy(lambda x: x[0])
V_RDD = business_RDD.map(lambda x: (x[0], [1 for _ in range(rank)]))#.sortBy(lambda x: x[0])

# U_RDD.take(5)
# V_RDD.take(5)


In [ ]:
review_overall_bis = review_overall_RDD.map(lambda x: (x[1], x[2], x[3]))
#review_overall_bis = review_RDD.map(lambda x: (x[1], x[2], x[3]))

#print(review_overall_RDD.filter(lambda x: x[1]=='cxuxXkcihfCbqt5Byrup8Q').first())

In [ ]:
#NOTE We are bypassing at the moment the validation set (useful for achieving the best rank)
trainingRDD, testRDD = review_overall_bis.randomSplit([7,3], seed=0)

In [ ]:
M_BC = sc.broadcast(trainingRDD.collect())

# list(filter(lambda x: x[0]=='cxuxXkcihfCbqt5Byrup8Q', M_BC.value))


In [ ]:
# todo no map
# V_BC = sc.broadcast(dict(V_RDD.map(lambda x : (x[0], x[1])).collect()))
V_BC = sc.broadcast(dict(V_RDD.collect()))
#V_BC.value['KsFuzQCOPhP8eHmt3rP11Q']



In [ ]:
# todo no map
# U_BC = sc.broadcast(dict(U_RDD.map(lambda x : (x[0], x[1])).collect()))
U_BC = sc.broadcast(dict(U_RDD.collect()))

#len(U_BC.value)


## Performing Optimization

In [ ]:
# normalized-->tuple: (user_id, buisness_id, overall - normalized_value)
# review_overall --> tuple: (review_id, user_id, business_id, stars + (partial overall + fans/best_fans))
#print(business_RDD.filter(lambda x: x[0]=='1H8ReY5GlGcHJz7umVidkg').first())

def update_U(entry):
    id = entry[0]
    U_value = entry[1]
    reviews = [i for i in M_BC.value if i[0] == id]
    for s in range(rank):
        res = 0
        den = 0
        for review in reviews:
            j = review[1] # business_id
            p = 0
            for k in [i for i in range(rank) if i != s]:
                #print(V_BC.value)
                
                p += U_value[k] * V_BC.value[j][k]

            res += V_BC.value[j][s] * (review[2] - p)
            den += V_BC.value[j][s]**2
        U_value[s] = res / (den if den > 0 else 1)\
            #if len(reviews)>0 else 1

    return (id, U_value)


In [ ]:
def update_V(entry):
    id = entry[0]
    V_value = entry[1]
    reviews = [i for i in M_BC.value if i[1] == id]

    # out = [1]*rank
    for s in range(rank):
        res = 0
        den = 0
        for review in reviews:
            j = review[0] # user_id
            p = 0
            for k in [i for i in range(rank) if i != s]:
                p += V_value[k] * U_BC.value[j][k]

            res += U_BC.value[j][s] * (review[2] - p)
            den += U_BC.value[j][s]**2
        V_value[s] = (res / (den if den > 0 else 1))\
#            if len(reviews)>0 else 1

    return (id, V_value)


# V_RDD.map(lambda x :  update_V(x)).take(20)

In [ ]:
def train(U_RDD,V_RDD,iterations):  
# iterations = 2
    global V_BC, U_BC
    for i in range(iterations):
        V_BC = sc.broadcast(dict(V_RDD.collect()))
        U_RDD = U_RDD.map(update_U)

        U_BC = sc.broadcast(dict(U_RDD.collect()))
        V_RDD = V_RDD.map(update_V)

    return U_RDD.collect(), V_RDD.collect()
    

In [ ]:
U,V = train(U_RDD, V_RDD, iterations =2)

In [ ]:
U

In [ ]:
V

In [ ]:
#Take an user-id and suggest business that he has not reviewed

def predict(u_value, business_value):
    return sum(u_value[i]* business_value[i] for i in range(len(u_value)))


def print_prediction(username, business_name):

    U_dict = dict(U)
    V_dict = dict(V)
    un = user_RDD.filter(lambda x: x[0]== username).first()[1]
    bn = business_RDD.filter(lambda x: x[0]== business_name).first()[2]
    print("User:",un, "----- Business:",bn, ":",predict(U_dict[username], V_dict[business_name]))



In [ ]:
print_prediction("DEFTCj_6zHCAtwyUvNEBZQ", "rBmpy_Y1UbBx8ggHlyb7hA")

In [ ]:
#RMSE of UV Decomposition

def rmse(testRDD):
    rmse = 0
    U_dict = dict(U)
    V_dict = dict(V)
    for i in testRDD.collect():
        rmse += (i[2] - predict(U_dict[i[0]], V_dict[i[1]]))**2
    return (rmse/len(M_BC.value))**0.5

print("RMSE:", rmse(testRDD))


Try to populate U and V with the average value of review rating

In [ ]:
avg_value = review_overall_bis.map(lambda x: (1, x[2])).reduceByKey(lambda x,y:(x+y)/2).first()[1]
avg_value

In [ ]:
U_RDD = user_RDD.map(lambda x: (x[0], [avg_value for _ in range(rank)]))#.sortBy(lambda x: x[0])
V_RDD = business_RDD.map(lambda x: (x[0], [avg_value for _ in range(rank)]))#.sortBy(lambda x: x[0])


In [ ]:
U,V = train(U_RDD, V_RDD, iterations =2)

In [ ]:
#RMSE of UV Decomposition
print("RMSE:", rmse(testRDD))


In [ ]:
import hashlib

def hash(s):
    h = hashlib.sha1(s.encode())
    return int(h.hexdigest(),16)%((2**31)-1)

review_overall_tris = review_overall_bis.map(lambda x: (hash(x[0]), hash(x[1]), x[2]))

In [ ]:
trainingRDD, validationRDD, testRDD = review_overall_tris.randomSplit([6, 2, 2], seed=0)

print('Training: %s, validation: %s, test: %s\n' % (trainingRDD.count(),
                                                    validationRDD.count(),
                                                    testRDD.count()))
print(trainingRDD.take(3))
print(validationRDD.take(3))
print(testRDD.take(3))

In [ ]:
import math

def computeError(predictedRDD, actualRDD):
    """ Compute the root mean squared error between predicted and actual
    Args:
        predictedRDD: predicted ratings for each movie and each user where each entry is in the form
                      (UserID, MovieID, Rating)
        actualRDD: actual ratings where each entry is in the form (UserID, MovieID, Rating)
    Returns:
        RSME (float): computed RSME value
    """
    # Transform predictedRDD into the tuples of the form ((UserID, MovieID), Rating)
    predictedReformattedRDD = predictedRDD.map(lambda i: ((i[0], i[1]), i[2]))

    # Transform actualRDD into the tuples of the form ((UserID, MovieID), Rating)
    actualReformattedRDD = actualRDD.map(lambda i: ((i[0], i[1]), i[2]))

    # Compute the squared error for each matching entry (i.e., the same (User ID, Movie ID) in each
    # RDD) in the reformatted RDDs using RDD transformtions - do not use collect()
    squaredErrorsRDD = (predictedReformattedRDD
                        .join(actualReformattedRDD)
                        .map(lambda i: math.pow(i[1][0] - i[1][1], 2))
                       )

    # Compute the total squared error - do not use collect()
    totalError = squaredErrorsRDD.reduce(lambda a, b: a+b)

    # Count the number of entries for which you computed the total squared error
    numRatings = squaredErrorsRDD.count()

    # Using the total squared error and the number of entries, compute the RSME
    return math.pow(float(totalError) / numRatings, 0.5)

In [ ]:
from pyspark.mllib.recommendation import ALS

validationForPredictRDD = validationRDD.map(lambda i: (i[0], i[1]))

seed = 5
iterations = 5
regularizationParameter = 0.1
ranks = [2, 4, 8, 12]
errors = [0, 0, 0, 0]
err = 0
tolerance = 0.02

minError = float('inf')
bestRank = -1
bestIteration = -1
for rank in ranks:
    model = ALS.train(trainingRDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularizationParameter)
    predictedRatingsRDD = model.predictAll(validationForPredictRDD)
    error = computeError(predictedRatingsRDD, validationRDD)
    errors[err] = error
    err += 1
    print('For rank {} the RMSE is {}'.format(rank, error))
    if error < minError:
        minError = error
        bestRank = rank

print('The best model was trained with rank {}'.format(bestRank))